In [ ]:
#https://github.com/alexeyev/abae-pytorch 

# TODO pulisci questo file
# todo  prova con is_alpha in preprocessing e buttiamo via il resto (con sequenze piu lunghe)!
#  --> leggi top 1000 di restaruant e vedi split con '' quanto lunghe in media
# todo fix regularization sia su matrix che generation di aspects / emb

# https://arxiv.org/pdf/1803.09820
# https://machinelearningmastery.com/learning-rate-for-deep-learning-neural-networks/

In [ ]:
# Una considerazione: Potrei sfruttare gli aspetti inferiti da quello che ho per filtrare via dal testo documenti che parlano di "Piattaforme", "Kickstarter", "Materiali" visto che queste le becca con facilita.
# Il mio dataset é probabilmente troppo vario al momento. Un filtraggio di questo tipo faciliterebbe l'apprendimento.

# Filtro a mano i commenti scaricati per togliere quelli poco significativi? Riduco rumore!

## Regularization
>We hope to learn vector representations of the most representative aspects for a review dataset. <br>
However, the aspect embedding matrix T may suffer from redundancy problems during training. [...] <br>
> The regularization term encourages orthogonality among the rows of the aspect embedding matrix T and penalizes redundancy between different aspect vectors <br>
> ~ Ruidan

We use an Orthogonal Regulizer definition of the method can be found here: https://paperswithcode.com/method/orthogonal-regularization. <br/>
For the code we use the default implementation provided by Keras (https://keras.io/api/layers/regularizers/)

## Hands on first attempt:


In [ ]:
# Choice of max_seq size is relative to the ds. 95th percentile (at max 5% loss of information)

In [1]:
from core.dataset import PandasPositiveNegativeNumericTextDataset
from core.train import ABAEModelConfiguration, ABAEModelManager
import pandas as pd

corpus = "../output/dataset/default-pre-processed/80k.preprocessed.csv"
config = ABAEModelConfiguration(corpus_file=corpus, model_name=f"ff")
config.epochs = 1  # Just one epoch
config.max_sequence_length = 40
config.aspect_size = 16
config.negative_sample_size = 20
print(f"Running on default config:\n {config}")

# Without any hp tuning we just try and see how it goes.
manager = ABAEModelManager(config)
train_dataset = PandasPositiveNegativeNumericTextDataset(
    dataframe=pd.read_csv(corpus),
    vocabulary=manager.embedding_model.vocabulary(),
    negative_size=config.negative_sample_size,
    max_seq_length=config.max_sequence_length
)

Running on default config:
 ABAEModelConfiguration(corpus_file='../output/dataset/default-pre-processed/80k.preprocessed.csv', model_name='ff', aspect_size=16, max_vocab_size=None, embedding_size=200, learning_rate=0.01, decay_rate=0.95, momentum=0.9, max_sequence_length=40, negative_sample_size=20, batch_size=64, epochs=1, output_path='./output')


Pandas Apply:   0%|          | 0/94248 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/94248 [00:00<?, ?it/s]

INFO:gensim.models.word2vec:collecting all words and their counts
INFO:gensim.models.word2vec:PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #10000, processed 82472 words, keeping 4112 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #20000, processed 168974 words, keeping 4464 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #30000, processed 252543 words, keeping 4521 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #40000, processed 335541 words, keeping 4536 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #50000, processed 419176 words, keeping 4538 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #60000, processed 504237 words, keeping 4538 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #70000, processed 587063 words, keeping 4543 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #80000, processed 672240 words, keeping 4543 word

Creating new model


INFO:gensim.models.word2vec:estimated required memory for 4544 words and 200 dimensions: 9542400 bytes
INFO:gensim.models.word2vec:resetting layer weights
INFO:gensim.utils:Word2Vec lifecycle event {'update': False, 'trim_rule': 'None', 'datetime': '2025-02-05T00:26:18.806964', 'gensim': '4.3.3', 'python': '3.12.6 (tags/v3.12.6:a4a2d2b, Sep  6 2024, 20:11:23) [MSC v.1940 64 bit (AMD64)]', 'platform': 'Windows-11-10.0.26100-SP0', 'event': 'build_vocab'}
INFO:gensim.utils:Word2Vec lifecycle event {'msg': 'training model with 3 workers on 4544 vocabulary and 200 features, using sg=1 hs=0 sample=0.001 negative=5 window=5 shrink_windows=True', 'datetime': '2025-02-05T00:26:18.807464', 'gensim': '4.3.3', 'python': '3.12.6 (tags/v3.12.6:a4a2d2b, Sep  6 2024, 20:11:23) [MSC v.1940 64 bit (AMD64)]', 'platform': 'Windows-11-10.0.26100-SP0', 'event': 'train'}
DEBUG:gensim.models.word2vec:job loop exiting, total 80 jobs
DEBUG:gensim.models.word2vec:worker exiting, processed 27 jobs
DEBUG:gensim.mo

Creating new model


Pandas Apply:   0%|          | 0/94248 [00:00<?, ?it/s]

Max sequence length calculation in progress...
Max sequence length is:  416 . The limit is set to 40 tokens.
We loose information on 166 points.This is 0.17613105848399965% of the dataset.
Padding sequences to length (40).


In [3]:
manager.run_train_process(train_dataset, optimizer='adam')

1473/1473 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step - loss: 12.9852 - max_margin_loss: 12.9535

DEBUG:h5py._conv:Creating converter from 5 to 3


1473/1473 ━━━━━━━━━━━━━━━━━━━━ 223s 151ms/step - loss: 12.9837 - max_margin_loss: 12.9520


(<keras.src.callbacks.history.History at 0x1ffb5708f50>,
 <Functional name=functional_2, built=True>)

In [42]:
iteration_model = manager.get_training_model()

In [43]:
import torch
# THIS IS OKAY!
from gensim import corpora
# See if another coherence metric might be better.
from core.evaluation import coherence_model_generation, get_aspect_top_k_words
from core.dataset import TokenizedDataset
from core.train import ABAEModelManager, ABAEModelConfiguration
from keras import ops as K

# word_emb = normalize(iteration_model.get_layer('word_embedding').weights[0].value.data)
# aspect_embeddings = normalize(iteration_model.get_layer('aspect_embedding').w)
word_emb = iteration_model.get_layer('word_embeddings').weights[0].value.data
aspect_embeddings = iteration_model.get_layer('aspect_embeddings').w

word_emb = torch.nn.functional.normalize(word_emb, p=2, dim=-1)
aspect_embeddings = torch.nn.functional.normalize(aspect_embeddings, p=2, dim=-1)

inv_vocab = manager.embedding_model.model.wv.index_to_key

aspects_top_k_words = [get_aspect_top_k_words(a, word_emb, inv_vocab, top_k=50) for a in aspect_embeddings]
aspect_words = [[word[0] for word in aspect] for aspect in aspects_top_k_words]  # Remap

ds = TokenizedDataset(corpus, manager.embedding_model.vocabulary())
dictionary = corpora.Dictionary(ds.text_ds.apply(lambda x: x.split(' ')).to_list())

_, m = coherence_model_generation(aspect_words, ds.text_ds.apply(lambda x: x.split(' ')), dictionary, topn=3)

m.get_coherence()


Given aspect most representative words are:
Word:  chapter (0.4432419240474701)
Word:  mistake (0.4312449097633362)
Word:  error (0.3956793248653412)
Word:  target (0.3923909664154053)
Word:  forum (0.38552427291870117)
Word:  playthrough (0.38476938009262085)
Word:  spoiler (0.378058522939682)
Word:  halfway (0.37712612748146057)
Word:  repair (0.3751535415649414)
Word:  reach (0.3593621850013733)
Word:  answer (0.3591642677783966)
Word:  spawn (0.355338454246521)
Word:  stop (0.34936630725860596)
Word:  passage (0.3478010296821594)
Word:  tank (0.3472113609313965)
Word:  punish (0.33871927857398987)
Word:  lidless (0.3372008800506592)
Word:  finish (0.3365933895111084)
Word:  raid (0.33627763390541077)
Word:  correct (0.3348654508590698)
Word:  impossible (0.33347582817077637)
Word:  correctly (0.33326420187950134)
Word:  faq (0.3319393992424011)
Word:  manual (0.32983875274658203)
Word:  sight (0.3298065662384033)
Word:  failure (0.32836371660232544)
Word:  runaway (0.3264343142509

Pandas Apply:   0%|          | 0/94248 [00:00<?, ?it/s]

Max sequence length calculation in progress...
Max sequence length is:  416 . The limit is set to 80 tokens.
We loose information on 21 points.This is 0.022281639928698752% of the dataset.
Padding sequences to length (80).


INFO:gensim.corpora.dictionary:adding document #0 to Dictionary<0 unique tokens: []>
INFO:gensim.corpora.dictionary:adding document #10000 to Dictionary<7882 unique tokens: ['game', 'german', 'involve', 'long', 'politic']...>
INFO:gensim.corpora.dictionary:adding document #20000 to Dictionary<11292 unique tokens: ['game', 'german', 'involve', 'long', 'politic']...>
INFO:gensim.corpora.dictionary:adding document #30000 to Dictionary<13709 unique tokens: ['game', 'german', 'involve', 'long', 'politic']...>
INFO:gensim.corpora.dictionary:adding document #40000 to Dictionary<15621 unique tokens: ['game', 'german', 'involve', 'long', 'politic']...>
INFO:gensim.corpora.dictionary:adding document #50000 to Dictionary<17363 unique tokens: ['game', 'german', 'involve', 'long', 'politic']...>
INFO:gensim.corpora.dictionary:adding document #60000 to Dictionary<18957 unique tokens: ['game', 'german', 'involve', 'long', 'politic']...>
INFO:gensim.corpora.dictionary:adding document #70000 to Diction

-0.09092904916702085

In [44]:
m.get_coherence_per_topic()

[-0.26420369098348445,
 -0.14356760386375775,
 -0.03789819035103982,
 -0.2244762502671176,
 0.27837966425198574,
 -0.19413690041888643,
 -0.2332492330429037,
 -0.29038464030316125,
 0.13041520886622168,
 0.06983114444193494]

In [45]:
from core.evaluation import coherence_per_aspect

aspect_words = [[word[0] for word in aspect] for aspect in aspects_top_k_words]
c, m = coherence_per_aspect(aspect_words, ds.text_ds, 3)

INFO:gensim.topic_coherence.text_analysis:CorpusAccumulator accumulated stats from 1000 documents
INFO:gensim.topic_coherence.text_analysis:CorpusAccumulator accumulated stats from 2000 documents
INFO:gensim.topic_coherence.text_analysis:CorpusAccumulator accumulated stats from 3000 documents
INFO:gensim.topic_coherence.text_analysis:CorpusAccumulator accumulated stats from 4000 documents
INFO:gensim.topic_coherence.text_analysis:CorpusAccumulator accumulated stats from 5000 documents
INFO:gensim.topic_coherence.text_analysis:CorpusAccumulator accumulated stats from 6000 documents
INFO:gensim.topic_coherence.text_analysis:CorpusAccumulator accumulated stats from 7000 documents
INFO:gensim.topic_coherence.text_analysis:CorpusAccumulator accumulated stats from 8000 documents
INFO:gensim.topic_coherence.text_analysis:CorpusAccumulator accumulated stats from 9000 documents
INFO:gensim.topic_coherence.text_analysis:CorpusAccumulator accumulated stats from 10000 documents
INFO:gensim.topic_c

In [46]:
c

[-15.415429966180449,
 -8.324924786749554,
 -11.348559564460805,
 -9.810242581716443,
 -3.3119131673751063,
 -14.003766614120599,
 -15.635291474652163,
 -15.271609512678603,
 -3.005173628424085,
 -5.745478321201425]

In [47]:
m.get_coherence()

-10.187238961755924

In [ ]:
# THIS IS OKAY!
from gensim import corpora
# See if another coherence metric might be better.
from core.evaluation import coherence_model_generation, get_aspect_top_k_words
from core.dataset import TokenizedDataset
from core.train import ABAEModelManager, ABAEModelConfiguration

c_file = "../output/dataset/pre-processed/tuning.preprocessed.csv"
# Load aspects
config = ABAEModelConfiguration(corpus_file=corpus, model_name=f"hands_on")

manager = ABAEModelManager(config)
iteration_model = manager.get_inference_model()

# word_emb = normalize(iteration_model.get_layer('word_embedding').weights[0].value.data)
# aspect_embeddings = normalize(iteration_model.get_layer('aspect_embedding').w)
word_emb = iteration_model.get_layer('word_embedding').weights[0].value.data
aspect_embeddings = iteration_model.get_layer('aspect_embedding').w

inv_vocab = manager.embedding_model.model.wv.index_to_key

aspects_top_k_words = [get_aspect_top_k_words(a, word_emb, inv_vocab, top_k=3) for a in aspect_embeddings]
aspect_words = [[word[0] for word in aspect] for aspect in aspects_top_k_words]  # Remap

ds = TokenizedDataset(c_file, manager.embedding_model.vocabulary())
dictionary = corpora.Dictionary(ds.text_ds.apply(lambda x: x.split(' ')).to_list())

_, m = coherence_model_generation(aspect_words, ds.text_ds.apply(lambda x: x.split(' ')), dictionary, topn=3)

m.get_coherence()

## Aspect Embedding Size
The aspect embedding size is what will be inferring aspects. It is closest to representative words (?). <br />
We have to identify 7 actual aspects (luck, bookkeeping, downtime...) but that does not mean our matrix should be limited to rows only! <br>

For the first try we setup the aspect_size:
>The optimal number of rows is problem-dependent, so it’s crucial to: <br/>
> Start with a heuristic: Begin with 2–3x the number of aspects.

For **aspect extraction**, which involves identifying key aspects or topics in text, the best early stopping method depends on your approach:

### 1. Embedding-based Methods (e.g., Clustering Embeddings)
- **Silhouette Score**: Measure the separation and compactness of clusters. Stop when the score stabilizes.
- **Inertia/Distortion**: Track the sum of squared distances within clusters and stop when improvement flattens.
- **Centroid Movement**: Stop when the change in cluster centroids across iterations is minimal.

### 2. Topic Modeling (e.g., LDA)
- **Perplexity**: Monitor the perplexity on a held-out dataset and stop when it stops decreasing significantly.
- **Coherence Score**: Measure the semantic consistency of extracted topics and stop when it stabilizes.

### 3. Autoencoder-based Aspect Extraction
- **Reconstruction Loss**: Stop training when the validation reconstruction error no longer improves.

### 4. Qualitative Evaluation (if feasible)
- Periodically inspect extracted aspects for meaningfulness and diversity to decide on stopping.

For **aspect extraction**, combining an automated metric (like coherence score or silhouette score) with manual inspection often yields the best results.


## Hyperparameters Tuning
To tune our parameters we use a filtered version of the 50k ds. <br>
We filter out rows that can be found on the 200k ds.

In [ ]:
import pandas as pd

# This is based on the idea that our dataset are generated with different seeds else it won't work
large = pd.read_csv("../output/dataset/pre-processed/200k.preprocessed.csv")
small = pd.read_csv("../output/dataset/pre-processed/100k.preprocessed.csv")
tuning_set = small[~small["comments"].isin(large["comments"])]

tuning_set.to_csv("../output/dataset/pre-processed/tuning.preprocessed.csv", index=False)

> The main goal of ABAE is to extract interpretable and meaningful aspects, which makes coherence the more aligned metric.<br> Reconstruction error might help guide training but doesn’t guarantee that the extracted aspects are semantically useful.

In [ ]:
from core.hp_tuning import ABAERandomHyperparametersSelectionWrapper, HyperparameterTuningManager

configurations = 4  # We try 15 different configurations
corpus_file = "../output/dataset/pre-processed/tuning.preprocessed.csv"

print(f"Starting procedure. We try a total of {configurations}")
hp_wrapper = ABAERandomHyperparametersSelectionWrapper.create()
hp_tuning_manager = HyperparameterTuningManager(hp_wrapper, corpus_file, "./output")

In [ ]:
hp_tuning_manager(different_configurations=configurations, repeat=3) # todo tune not on metric but on loss. We want the best reconstruciton model.
# Bottom line: Prioritize classification performance, but monitor coherence if it affects usability.
# The output of this cell is very long therefore I deleted it

In [ ]:
from pathlib import Path
import json
import pandas as pd
import numpy as np

dataframe = pd.DataFrame(
    columns=[
        'name', 'mean_loss', 'mean_coherence', 'aspect_size', 'embedding_size', 'epochs', 'batch_size', 'learning_rate',
        'decay_rate', 'momentum', 'negative_sample_size'
    ]
)

i = 0
# Inspect results we stored for process. todo: Quando finisci salvalo come file da "tenere" in repo cosi che chi controlla lo ha gia.
for element in Path("./output").iterdir():

    # Search only for the tuning process runs.
    if element.is_dir() and element.name.startswith("tuning"):
        run_result = json.load(open(str(element) + "/run_results.json"))
        # We want the best configurations.

        obj = {
            'name': element.name,
            'mean_loss': np.mean(run_result["evaluation_loss"]),
            'mean_coherence': np.mean(run_result["coherence"])
        }
        temp = pd.DataFrame(run_result['params'] | obj, index=[i])
        dataframe = pd.concat([temp, dataframe])
        i += 1

In [ ]:
ds.text_ds.apply(lambda x: x.split(' '))

In [ ]:
# THIS IS OKAY!

from gensim import corpora
# See if another coherence metric might be better.
from core.evaluation import coherence_model_generation, normalize, get_aspect_top_k_words
import json
from core.dataset import TokenizedDataset
from core.train import ABAEModelManager, ABAEModelConfiguration

model_name = 'tuning_7128da5c-04c0-4e91-abf2-462f9646b48d'
c_file = "../output/dataset/pre-processed/tuning.preprocessed.csv"
# Load aspects
parameters = json.load(open(f"./output/{model_name}/run_results.json"))['params']
config = ABAEModelConfiguration(corpus_file=c_file, model_name=model_name, **parameters)

manager = ABAEModelManager(config)
iteration_model = manager.get_inference_model()

word_emb = normalize(iteration_model.get_layer('word_embedding').weights[0].value.data)
aspect_embeddings = normalize(iteration_model.get_layer('aspect_embedding').w)

# word_emb = iteration_model.get_layer('word_embedding').weights[0].value.data
# aspect_embeddings = iteration_model.get_layer('aspect_embedding').w

inv_vocab = manager.embedding_model.model.wv.index_to_key

aspects_top_k_words = [get_aspect_top_k_words(a, word_emb, inv_vocab, top_k=20) for a in aspect_embeddings]
aspect_words = [[word[0] for word in aspect] for aspect in aspects_top_k_words]  # Remap

ds = TokenizedDataset(c_file, manager.embedding_model.vocabulary())
dictionary = corpora.Dictionary(ds.text_ds.apply(lambda x: x.split(' ')).to_list())

_, m = coherence_model_generation(aspect_words, ds.text_ds.apply(lambda x: x.split(' ')), dictionary, topn=10)

m.get_coherence()

In [ ]:
manager.embedding_model.model.wv

In [ ]:
len(ds)

In [52]:
from torch.utils.data import DataLoader
from core.dataset import SimpleWord2VecEmbeddingsDataset

iteration_model = manager.get_inference_model()
custom_review = [
    'game hard fun', # 0
    'brother forever decide',
    'zero luck all skill',
    'lot downtime', # 3
    'i have banana',
    'plastic inserts',
    'complex game rules' # 6
]
# todo rebuild a prediciton model senza negative predict. Devo ristrutturare tutto1
ds = SimpleWord2VecEmbeddingsDataset(custom_review, manager.embedding_model.model)

res = iteration_model.predict(DataLoader(ds, batch_size=32))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step


In [54]:
res[0]

array([[1.0000000e-07, 1.0000000e-07, 1.0000000e-07, 1.0000000e-07,
        1.0000000e-07, 1.0000000e-07, 1.0000000e-07, 1.0000000e-07,
        1.0000000e-07, 1.0000000e-07, 1.0000000e-07, 1.0000000e-07,
        1.0000000e-07, 1.0000000e-07, 1.0000000e-07, 1.0000000e-07,
        1.0000000e-07, 1.0000000e-07, 1.0000000e-07, 1.0000000e-07,
        1.0000000e-07, 1.0000000e-07, 1.0000000e-07, 1.0000000e-07,
        1.0000000e-07, 1.0000000e-07, 1.0000000e-07, 1.0000000e-07,
        1.0000000e-07, 1.0000000e-07, 1.0000000e-07, 1.0000000e-07,
        1.0000000e-07, 1.0000000e-07, 1.0000000e-07, 1.0000000e-07,
        1.0000000e-07, 1.6263971e-01, 1.6070150e-01, 6.7665911e-01],
       [1.0000000e-07, 1.0000000e-07, 1.0000000e-07, 1.0000000e-07,
        1.0000000e-07, 1.0000000e-07, 1.0000000e-07, 1.0000000e-07,
        1.0000000e-07, 1.0000000e-07, 1.0000000e-07, 1.0000000e-07,
        1.0000000e-07, 1.0000000e-07, 1.0000000e-07, 1.0000000e-07,
        1.0000000e-07, 1.0000000e-07, 1.0000000

In [53]:
# Il modello fa schifo non serve a niente.
res[1]  # Riddle correctly yields a classification to the aspect for representative.

array([[0.0817567 , 0.1313807 , 0.05315639, 0.08426162, 0.12962303,
        0.08049816, 0.11351769, 0.10415612, 0.10272776, 0.11892185],
       [0.10805491, 0.07200906, 0.09453166, 0.10224397, 0.09893427,
        0.07725929, 0.08951658, 0.13686667, 0.08059887, 0.13998476],
       [0.08384306, 0.09782003, 0.10227967, 0.09017856, 0.08234558,
        0.1649447 , 0.09660699, 0.11256695, 0.08123559, 0.08817887],
       [0.10481264, 0.15214735, 0.0577409 , 0.08501475, 0.05284812,
        0.07585361, 0.12177281, 0.13394837, 0.10639574, 0.10946572],
       [0.07773   , 0.08794104, 0.12174451, 0.08094119, 0.07236784,
        0.1945742 , 0.09200998, 0.10570648, 0.07923861, 0.08774615],
       [0.08017664, 0.06221417, 0.12141559, 0.10387982, 0.0707244 ,
        0.1540822 , 0.08008087, 0.09180547, 0.1505977 , 0.08502316],
       [0.11342146, 0.12564196, 0.09231173, 0.07948986, 0.08358911,
        0.09293932, 0.08486223, 0.08784374, 0.13912956, 0.10077106]],
      dtype=float32)

In [ ]:
m.get_coherence_per_topic()

In [ ]:
len(ds.dataset.to_list())
list(filter(lambda x: x.any() is None or len(x) < 1, ds.dataset.to_list()))

# Focus on learning rate

In [ ]:
# We fix other params and now focus entirely on lr.
# We have already a "promising" range defined.
# We look in that space so we redefine lr on ABAERandomHyperparametersSelectionWrapper

## Best found model training:

## See if the Hp tuning really improved upon our results:
We used SGD anda learned its parameters under the assumption that we would do better. <br>
Let's see if it really is the case, or we just wasted time.

For comparison we use Adam that has the advantage of being robust enough without parameter scouting.

In [ ]:
#todo

## Test accuracy on small test sample we filled out

### Test set definition

In [ ]:
from core.pre_processing import PreProcessingService

# We take around 1k records that were not seen yet from the model and label them by hand.
dataset = pd.read_csv("../data/corpus.csv")
# Handle game names. Or not? I don't need the full pipeline I guess. todo
game_names = pd.read_csv("../resources/2024-08-18.csv")['Name']
game_names = pd.concat([game_names, pd.Series(["Quick", "Catan"])], ignore_index=True)
document_game_names = game_names.swifter.apply(lambda x: nlp(x)).tolist()

pipeline = PreProcessingService.full_pipeline(document_game_names, "../data/processed-dataset/full")

# Extract 1k from dataset that are not in 200k
train_ds = pd.read_csv("../output/dataset/pre-processed/200k.preprocessed.csv")

# Take top 2k. (We will select some good ones and reduce the number to 1k)
test_set = dataset[~dataset["comments"].isin(train_ds["comments"])]

We have to use labels:

In [ ]:
labels = {
    '0': "Luck/Alea",
    '1': 'Bookkeeping',
    '2': 'Downtime',
    '3': 'Interaction',
    '4': 'Bash',
    '5': 'Complicated/Complex',  # I could watch weight to see if there is a ratio relation.
    '6': 'Misc'
}